In [ ]:
# 在AWS SageMaker > Notebook > 開啟Jupyter > 右鍵 New Notebook > 切換Kernel為'Conda_pytorch'
!python --version

Python 3.10.19


In [2]:
import sys
!{sys.executable} -m pip install protobuf==4.25.8
!{sys.executable} -m pip install onnx==1.19.1 onnxslim==0.1.82

In [3]:
import sys
print(sys.executable)

# -----------------------------------------------

import google.protobuf
import onnx
print(google.protobuf.__file__)
print(google.protobuf.__version__)
print("onnx:", onnx.__version__)

/home/ec2-user/anaconda3/envs/pytorch/bin/python
/home/ec2-user/anaconda3/envs/pytorch/lib/python3.10/site-packages/google/protobuf/__init__.py
4.25.8
onnx: 1.19.1


In [4]:

from ultralytics import YOLO

# 1. 載入模型
model = YOLO("best.pt")

# 2. 匯出 ONNX（ESP32-P4 相容設定）
model.export(
    format="onnx",
    imgsz=320,        # 固定輸入尺寸（重要）
    opset=11,         # ESP-DL 支援
    simplify=True,    # ONNX Graph 簡化
    dynamic=False     # 一定要關
)

print("ONNX export done.")

Ultralytics 8.3.246 🚀 Python-3.10.19 torch-2.7.0+cu128 CPU (Intel Xeon Platinum 8124M CPU @ 3.00GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 5, 2100) (5.2 MB)

ONNX: starting export with onnx 1.19.1 opset 11...
ONNX: slimming with onnxslim 0.1.82...


2026-01-03 17:10:04.384877257 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


ONNX: export success ✅ 1.1s, saved as 'best.onnx' (10.0 MB)

Export complete (1.4s)
Results saved to /home/ec2-user/SageMaker/esp/113368507
Predict:         yolo predict task=detect model=best.onnx imgsz=320  
Validate:        yolo val task=detect model=best.onnx imgsz=320 data=/tmp/data/data.yaml  
Visualize:       https://netron.app
ONNX export done.


In [5]:
import onnx

# 檢查onnx
m = onnx.load("best.onnx")
print(m.graph.input[0].type.tensor_type.shape)

dim {
  dim_value: 1
}
dim {
  dim_value: 3
}
dim {
  dim_value: 320
}
dim {
  dim_value: 320
}



In [6]:
for o in m.graph.output:
    dims = o.type.tensor_type.shape.dim
    print(o.name, [d.dim_value for d in dims])

output0 [1, 5, 2100]
